In [148]:
import dendropy
from dendropy.simulate import treesim
from dendropy.calculate import treecompare
import random

In [266]:
def print_generation(n):
    familly['F{}L_filtered'.format(n)].print_plot()
    print(familly['F{}L_mutation'.format(n)])

    familly['F{}R_filtered'.format(n)].print_plot()
    print(familly['F{}R_mutation'.format(n)])


    print(familly['F{}_URF'.format(n)])

In [318]:
forest = []
size_forest = 100
n_generations = 4

for e in range(size_forest):
    
    to_store = {}
    
        
    familly = {}
    familly['tree_size'] = e+4
    familly['tns'] = dendropy.TaxonNamespace(['T_{}'.format(i) for i in range(1,int(e * 1.5))])

    # ROOT PAIR
    familly['L'] = treesim.birth_death_tree(1, 0,taxon_namespace=familly['tns'], num_extant_tips=familly['tree_size'])
    familly['R'] = treesim.birth_death_tree(1, 0,taxon_namespace=familly['tns'], num_extant_tips=familly['tree_size'])
    to_store['L'] = familly['L'].as_string(schema='newick')
    to_store['R'] = familly['R'].as_string(schema='newick')
    
    familly['common'] = set(n.taxon for n in familly['L'].leaf_node_iter()) & set(n.taxon for n in familly['R'].leaf_node_iter())
           
    familly['L_filter'] = familly['L'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    familly['L_filter'].is_rooted = False
    familly['R_filter'] = familly['R'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    familly['R_filter'].is_rooted = False
        
    to_store['root_URF'] = treecompare.symmetric_difference( familly['L_filter'], familly['R_filter'], is_bipartitions_updated=True)
    to_store['root_WRF'] = treecompare.weighted_robinson_foulds_distance(familly['L_filter'], familly['R_filter'], is_bipartitions_updated=True)
            
    # NEXT GENERATIONS
    
    parent_L = familly['L']
    parent_R = familly['R']
    for F in range(1,n_generations):
        
        def build_child(label_string, parent_, force= False):
            
            mutate = False
            
            familly[label_string] = parent_.extract_tree()

            if force or random.random() > 0.5:
                n = random.choice(list(familly[label_string].preorder_node_iter())[1:])
                familly[label_string].reroot_at_edge(n.edge,length1=0.5 * n.edge_length, 
length2=0.5 * n.edge_length, update_bipartitions=True)
                to_store[label_string + '_mutation']  = [x.taxon.label for x in n.leaf_nodes()]
                mutate = True
                

            else:
                to_store[label_string + '_mutation']  = None
                
            
            familly[label_string + '_filtered'] = familly[label_string].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
            familly[label_string + '_filtered'].is_rooted = False
            return mutate
        
        
        rerooted = build_child('F{}L'.format(F),parent_L )
        if (rerooted):
            build_child('F{}R'.format(F), parent_R)
        else:
            build_child('F{}R'.format(F), parent_R, force = True)
        
            
        to_store['F{}_URF'.format(F)] = treecompare.symmetric_difference(familly['F{}L_filtered'.format(F)], familly['F{}R_filtered'.format(F)], is_bipartitions_updated=True)
        to_store['F{}_WRF'.format(F)] = treecompare.weighted_robinson_foulds_distance(familly['F{}L_filtered'.format(F)], familly['F{}R_filtered'.format(F)], is_bipartitions_updated=True)
                
        parent_L = familly['F{}L'.format(F)]
        parent_R = familly['F{}R'.format(F)]

    forest.append(to_store)
        



In [319]:
forest

[{'L': '[&R] (((T3:0,T4:0):1.1227186045685345,T2:1.1227186045685345):0.6624273338933107,T1:1.785145938461845):1.1029154690329714;\n',
  'R': '[&R] (((T4:0,T2:0):0.3810812621119617,T3:0.3810812621119617):0.11340239256911812,T1:0.4944836546810798):0.19312749021541356;\n',
  'root_URF': 2,
  'root_WRF': 5.757074937283508,
  'F1L_mutation': None,
  'F1R_mutation': ['T1'],
  'F1_URF': 2,
  'F1_WRF': 5.757074937283508,
  'F2L_mutation': ['T1'],
  'F2R_mutation': None,
  'F2_URF': 2,
  'F2_WRF': 5.757074937283508,
  'F3L_mutation': None,
  'F3R_mutation': ['T4', 'T2'],
  'F3_URF': 2,
  'F3_WRF': 5.757074937283508},
 {'L': '[&R] (((T1:0.24504960278940993,(T3:0,T5:0):0.24504960278940993):0.12638015876228786,T4:0.3714297615516978):0.14137085289550877,T2:0.5128006144472066):1.1060714776299476;\n',
  'R': '[&R] (((T5:0,T1:0):0.39352393749900894,T4:0.39352393749900894):0.7019218807473704,(T3:0.11150356814122649,T2:0.11150356814122649):0.9839422501051529):1.5107235920034108;\n',
  'root_URF': 4,
  '

In [321]:
import json

with open('distance_testing.json', 'w') as fp:
    json.dump(forest, fp)